In [2]:
import numpy as np
import functools as ft
import itertools
import os
from NestedForLoop import get_iterator


In [106]:
qubit_number=4
Pauli=np.asarray([
        [[1,0],
        [0,1]],


        [[0,1],
        [1,0]],


        [[0,-1j],
        [1j,0]],


        [[1,0],
        [0,-1]]])


states=np.asarray([
    [1,1]/np.sqrt(2),
    [1,-1]/np.sqrt(2),
    [1,1j]/np.sqrt(2),
    [1,-1j]/np.sqrt(2),
    [1,0],
    [0,1],

])

In [4]:
def get_proj_iterator(N_qubit):
    d={"x": [[1/np.sqrt(2),1/np.sqrt(2)],[1/np.sqrt(2),-1/np.sqrt(2)]], "y":[[1/np.sqrt(2),complex(0,1/np.sqrt(2))],[1/np.sqrt(2),complex(0,-1/np.sqrt(2))]], "z":[[1,0],[0,1]] }
    basis_order= get_iterator(3,N_qubit)
    basis=["x","y","z"]
    projectors_order=get_iterator(2,N_qubit)
    columns=[]
    
    
    character_Matrix=[]
    for i in basis_order:
        string=""
        for j in i:
            string=string+basis[j]
        lines=[]    
        for k in projectors_order:
            o=0
            vect=[]
            for n in k:
                
                vect.append(d[string[o]][n])
                o=o+1
            lines.append(vect)    
        character_Matrix.append(lines)         
        columns.append(string)   
        
    return character_Matrix 
v=  get_proj_iterator(4)





qubit_number=4
Pauli=np.asarray([
        [[1,0],
        [0,1]],


        [[0,1],
        [1,0]],


        [[0,-1j],
        [1j,0]],


        [[1,0],
        [0,-1]]])


states=np.asarray([
    [1,1]/np.sqrt(2),
    [1,-1]/np.sqrt(2),
    [1,1j]/np.sqrt(2),
    [1,-1j]/np.sqrt(2),
    [1,0],
    [0,1],

])



    
observables = []
perm = itertools.product(Pauli, repeat=qubit_number)
for i in list(perm):
    observables.append(ft.reduce(np.kron, i)) 

psi=np.zeros((len(observables[1:]),len(projectors2)))
# psi_aux=np.zeros((3*qubit_number,2**qubit_number,4*qubit_number-1))

for i in range(len(observables[1:])):
    for j in range(len(projectors2)):
        psi[i][j]=np.trace(projectors2[j]@observables[i+1])/(2**qubit_number)
psi_aux=np.reshape(np.transpose(psi),[3**qubit_number,2**qubit_number,4*qubit_number-1]) 

invPSI=np.linalg.inv(psi@psi.transpose())


MemoryError: Unable to allocate 16.0 GiB for an array with shape (512, 1048576, 2) and data type complex128

In [107]:
projectors = []
perm = itertools.product(states, repeat=qubit_number)
for i in list(perm):
    projectors.append(np.outer(ft.reduce(np.kron, i),np.conjugate(np.transpose(ft.reduce(np.kron, i)))))
    
observables = []
perm = itertools.product(Pauli, repeat=qubit_number)
for i in list(perm):
    observables.append(ft.reduce(np.kron, i))

In [108]:
psi=np.zeros((len(observables[1:]),len(projectors)))
psi_aux=np.zeros((3**qubit_number,2**qubit_number,4**qubit_number-1))

for i in range(len(observables[1:])):
    for j in range(len(projectors)):
        psi[i][j]=np.trace(projectors[j]@observables[i+1])/(2**qubit_number)
        a=int(j%2+2*((int(j/6)%(2**(qubit_number-1)))))
        b=int(j/2)%3
        c=3*int(j/(6*2**(qubit_number-1)))
#       print(i,j,b+c,a,i)
        psi_aux[int(b+c)][a][i]=np.trace(projectors[j]@observables[i+1])/(2**qubit_number)


C:\Users\nicol\AppData\Local\Temp\ipykernel_13340\3891056298.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
  psi[i][j]=np.trace(projectors[j]@observables[i+1])/(2**qubit_number)
C:\Users\nicol\AppData\Local\Temp\ipykernel_13340\3891056298.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  psi_aux[int(b+c)][a][i]=np.trace(projectors[j]@observables[i+1])/(2**qubit_number)


In [8]:
inv=np.linalg.inv(psi@np.transpose(psi))
            
working_dir = os.getcwd()
np.save(working_dir+'\SavedVariables\X_matrix'+str(qubit_number), psi_aux)
np.save(working_dir+'\SavedVariables\invXtX_matrix'+str(qubit_number), inv)

In [28]:
# m=np.load(working_dir+'\SavedVariables\X_matrix2.npy')
# print(np.round(psi_aux-m,3))